# StegaShield Profile Smoke Tests

Use this notebook to run a fast, deterministic check of every public StegaShield profile (robust, semi-fragile, hybrid). It mirrors the logic inside `tests/profile_smoke_test.py` so you can review metrics, heatmaps, and regression data inside Colab or your local IDE.


In [ ]:
# Optional: install project dependencies (uncomment for Colab)
# !pip install --quiet numpy pillow opencv-python pywavelets


In [ ]:
from pathlib import Path
import sys

possible_roots = [
    Path.cwd(),
    Path.cwd() / "stegashield_model",
    Path("/content/stegashield_model"),
    Path("/content/drive/MyDrive/stegashield_model"),
]

PROJECT_DIR = None
for root in possible_roots:
    if (root / "stegashield_profiles.py").exists():
        PROJECT_DIR = root
        break

if PROJECT_DIR is None:
    PROJECT_DIR = Path.cwd()
    print(f"⚠️ Could not auto-detect repo; defaulting to {PROJECT_DIR}")
else:
    print(f"✅ Using project dir: {PROJECT_DIR}")

if str(PROJECT_DIR) not in sys.path:
    sys.path.insert(0, str(PROJECT_DIR))



In [ ]:
from tests.profile_smoke_test import run_smoke_tests

run_smoke_tests()


In [ ]:
import json

summary_path = PROJECT_DIR / "tests" / "artifacts" / "smoke_summary.json"
if summary_path.exists():
    with open(summary_path, "r", encoding="utf-8") as f:
        summary = json.load(f)
    summary
else:
    print(f"❌ Summary file not found at {summary_path}. Run the previous cell first.")


## Hybrid Consistency Checks

The smoke runner embeds with the semi-fragile layer first, then feeds that intermediate result into the robust LSB layer. The verification report therefore should show:

- `semi_fragile_report.bit_accuracy` ≈ 1.0 even after the second embedding stage (confirms BER is stable)
- `heatmap_path` exported for both semi-fragile and hybrid modes (confirming visualization survives)
- `robust_report.verdict == "AUTHENTIC"`

If any of these regress, the test raises an `AssertionError` so CI / Colab stops immediately.


## Future: Fragile Profile

To spin up the stricter "fragile" preset, keep using the same wrapper API but instantiate the semi-fragile layer with tighter parameters before calling `embed_image(..., mode="fragile")`. Recommended tweaks (see README):

- Smaller DWT block size (4×4 or 2×2)
- Higher redundancy (≥5)
- Lower `q_step` (≈2.0) so any perturbation flips the quantization bin

Once you finalize those constants, re-run the smoke notebook to make sure the fragile preset fails fast (tamper detected) when you apply even tiny edits.


In [ ]:
from models.semi_fragile_dwt_svd import DwtSvdParams, SemiFragileEmbedderDwtSvd

fragile_params = DwtSvdParams(block_size=4, redundancy=5, q_step=2.0)
fragile_embedder = SemiFragileEmbedderDwtSvd(params=fragile_params)
fragile_params
